In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Data preprocessing

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

root_dir = "/content/drive/MyDrive/ewha/pattern"

# Checking if our specified directory exists
os.path.exists(root_dir)
file_name = "train.csv"

# Load the dataset using pandas
df= pd.read_csv(os.path.join(root_dir, file_name))
df.head()


##variable selection

In [ ]:
df = df.drop(columns='id')
df = df.drop(columns='contact')
df = df.drop(columns='pdays')
df = df.drop(columns='emp.var.rate')
df = df.drop(columns='day_of_week')
df = df.drop(columns="default")
df = df.drop(columns='cons.conf.idx')

In [ ]:
df.replace('unknown', np.nan, inplace=True)
df.replace('nonexistent', np.nan, inplace=True)

## Standardization and Normalization

* 규제의 효과 향상
* 거리기반모델(knn,svm),경사 하강법 기반 모델(선형회귀, 로지스틱회귀, 인공신경망)에 특성 스케일링의 영향을 받음


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Select numerical features
numeric_features = df.select_dtypes(include=['int', 'float']).columns

# Extract numerical data
numeric_data = df[numeric_features]

# Normalize numerical data
normalizer = MinMaxScaler()
normalized_data = normalizer.fit_transform(numeric_data)
normalized_data = pd.DataFrame(normalized_data, columns=numeric_features)

# Standardize normalized data , 스케일링
scaler = StandardScaler()
standardized_data = scaler.fit_transform(normalized_data)
standardized_data = pd.DataFrame(standardized_data, columns=numeric_features)

# Combine standardized numeric data with categorical data
processed_data = pd.concat([standardized_data, df.drop(columns=numeric_features)], axis=1)
df = processed_data

In [ ]:
df

In [ ]:
df["y"] = df["y"].replace({"no": 0, "yes": 1})

In [ ]:
label = ['education', 'housing', 'loan', 'month', 'poutcome']

In [ ]:
for col in label:
    if df[col].dtype == 'object':
        encoder = LabelEncoder()
        df[col] = encoder.fit_transform(df[col])

for col in label:
    if df_test[col].dtype == 'object':
        encoder = LabelEncoder()
        df_test[col] = encoder.fit_transform(df_test[col])

print(df)

2. OneHot Encoding

In [ ]:
cat = ['job', 'marital']

In [ ]:
encoded_df = pd.get_dummies(df, columns=cat)
encoded_test = pd.get_dummies(df_test, columns=cat)

encoded_df.replace({True: 1, False: 0}, inplace=True)
encoded_test.replace({True: 1, False: 0}, inplace=True) # Encode True and False as 1 and 0

In [ ]:
df = encoded_df
df_test = encoded_test

In [ ]:
df

*missing value*
- numerical은 예측 모델로 결측치 채우기
* categorical은 최빈값으로 결측치 채우기

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
import pandas as pd


# MICE를 사용하여 결측치를 대체합니다.
imputer = IterativeImputer()
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

df = df_imputed
df

outlier process
* 고차원 데이터는 isolation이 적합
outlier 인식 개수가 너무 차이나서 보수적으로 isolation 선택
* dbscan through pca :13, isolation :659

In [ ]:
from sklearn.ensemble import IsolationForest
random_state = 100
df_outlier = df.copy()
features = df.columns.drop('y')

iso_forest = IsolationForest(n_estimators=100, contamination=0.02, random_state=random_state)
outliers = iso_forest.fit_predict(df_outlier[features]) #1,-1 value

print("Outliers detected:", np.sum(outliers == -1))
df_outlier['outlier'] = outliers

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

def replace_outliers_with_model(df, features_df):
    # 이상치를 포함한 데이터 복사
    df_outliers = df.copy()

    # Isolation Forest 모델 생성 및 학습
    isolation_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=random_state)
    outliers = isolation_forest.fit_predict(df_outliers[features_df])

    # 예측을 위한 RandomForestRegressor 모델 생성
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

    for feature in features_df:
        # 이상치 인덱스 추출
        outlier_indices = df_outliers.index[outliers == -1]

        if len(outlier_indices) > 0:
            # 이상치를 제외한 데이터로 모델 학습
            train_data = df_outliers.drop(outlier_indices)
            rf_model.fit(train_data.drop(features_df, axis=1), train_data[feature])

            # 이상치 예측
            outlier_data = df_outliers.loc[outlier_indices].drop(features_df, axis=1)
            predicted_outliers = rf_model.predict(outlier_data)

            # 예측된 값으로 이상치 대체
            df_outliers.loc[outlier_indices, feature] = predicted_outliers

    return df_outliers

features_df = df.columns.drop('y')

# 이상치를 예측하고 예측된 값으로 대체
df = replace_outliers_with_model(df, features_df)

In [ ]:
df_outlier = df.copy()

iso_forest = IsolationForest(n_estimators=100, contamination=0.02, random_state=random_state) #contamination이 뭔지 확인
outliers = iso_forest.fit_predict(df_outlier[features]) #1,-1 value

print("Outliers detected:", np.sum(outliers == -1))
df_outlier['outlier'] = outliers

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

X = df.drop('y',axis=1)

# PCA로 차원 축소
pca = PCA(n_components=2)  # 필요한 차원 수로 설정
X_reduced = pca.fit_transform(X)

# K-means 클러스터링 적용
kmeans = KMeans(n_clusters=3, random_state=0)
clusters = kmeans.fit_predict(X_reduced)
df['cluster'] = clusters
df

data split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

shuffle = True
test_size_ratio = 0.18 # The optimal test dataset ratio is specified as 0.25.(by experience)

X = df.drop('y', axis =1)
y=pd.Series(df.y)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_ratio, random_state=random_state, shuffle=shuffle)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


Modeling

In [ ]:
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import f1_score

kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
scoring = "neg_mean_squared_error"

models = {}


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# 데이터 준비 (예제로 랜덤 데이터 생성)
# X_train, y_train, X_test, y_test 등의 데이터가 있다고 가정합니다.

# 모델 정의
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(26,)),  # 입력 레이어
    layers.Dense(32, activation='relu'),  # 은닉 레이어
    layers.Dense(2, activation='softmax')  # 출력 레이어 (다중 클래스 분류에서는 softmax 사용)
])

# 모델 컴파일
model.compile(optimizer='adam',  # 옵티마이저
              loss='sparse_categorical_crossentropy',  # 손실 함수 (다중 클래스 분류에서는 sparse_categorical_crossentropy 사용)
              metrics=['accuracy'])  # 평가 지표


# 모델 학습
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# 모델 평가
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print(f1)


In [ ]:
#decisionTree
model = DecisionTreeRegressor(random_state=random_state)

# Define the hyperparameters and their possible values
param_grid = {
    "max_depth": [5, 10, 20],
    "min_samples_split": [2, 10, 20],
    "ccp_alpha": [0.0, 0.01],
}

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

models["Decision Tree"] = grid_search.best_estimator_

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print(f1)

In [ ]:
# DecisionTree Classifier
model = DecisionTreeClassifier(random_state=13)

param_grid = {
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 10, 20],
    "min_samples_leaf": [1, 5, 10],
}

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring='accuracy', refit=True)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

models["Decision Tree"] = grid_search.best_estimator_

y_prob = grid_search.predict_proba(X_test)
y_pred =grid_search.predict(X_test)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print(f1)

"""print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob[:, 1]))"""

In [ ]:
# baggingRegressor
base_model = DecisionTreeRegressor()
model = BaggingRegressor(estimator=base_model,
                         bootstrap=True,
                         n_jobs=-1,
                         random_state=random_state)

# Define the hyperparameters and their possible values
param_grid = {
    "n_estimators": [25, 50]
}

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

models["Bagging"] = grid_search.best_estimator_

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print(f1)

In [ ]:
# Bagging Classifier
base_model = DecisionTreeClassifier()
model = BaggingClassifier(estimator=base_model,
                         bootstrap=True,
                         n_jobs=-1,
                         random_state=random_state)

# Define the hyperparameters and their possible values
param_grid = {
    "n_estimators": [25, 50]
}

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

models["Bagging"] = grid_search.best_estimator_

y_prob = grid_search.predict_proba(X_test)
y_pred = grid_search.predict(X_test)

"""print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob[:, 1]))"""

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print(f1)

In [ ]:
#randomforest
model = RandomForestRegressor(max_depth=None,
                              min_samples_split=2,
                              bootstrap=True,
                              n_jobs=-1,
                              random_state=random_state)

# Define the hyperparameters and their possible values
param_grid = {
    "n_estimators": [25, 50],
    "max_features": [0.5, "sqrt", "log2", None],
}

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

models["Random Forest"] = grid_search.best_estimator_

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print(f1)

In [ ]:
# AdaBoostClassifier 모델 생성
model = AdaBoostClassifier(random_state=random_state)

# 하이퍼파라미터 그리드 정의
param_grid = {
    "n_estimators": [25, 50],
    "base_estimator": [DecisionTreeClassifier(max_depth=3), DecisionTreeClassifier(max_depth=6)],
    "learning_rate": [0.1, 1.0],
}


grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

models["AdaBoost"] = grid_search.best_estimator_

y_prob = grid_search.predict_proba(X_test)
y_pred = grid_search.predict(X_test)

"""print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob[:, 1]))"""

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print(f1)

In [ ]:
#GB
model = GradientBoostingRegressor(loss="squared_error",
                                  subsample=1.0,
                                  random_state=random_state)

# Define the hyperparameters and their possible values
param_grid = {
    "n_estimators": [25, 50],
    "max_depth": [3, 6],
    "learning_rate": [0.0, 0.1],
}

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

models["Gradient Boosting"] = grid_search.best_estimator_

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print(f1)

In [ ]:
#XGB Regressor
model = XGBRegressor(subsample=1.0,
                     learning_rate=0.1,
                     max_depth=7,
                     n_jobs=-1,
                     random_state=random_state)

# Define the hyperparameters and their possible values
param_grid = {
    "n_estimators": [25, 50],
    "reg_alpha": [0, 0.1],
    "reg_lambda": [0, 0.1],
}

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit=True)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

models["XGBoost"] = grid_search.best_estimator_

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print(f1)

In [ ]:
# XGB classifier
model = XGBClassifier(subsample=1.0,
                     learning_rate=0.1,
                     max_depth=6,
                     n_jobs=-1,
                     random_state=random_state)

# Define the hyperparameters and their possible values
param_grid = {
    "n_estimators": [25, 50],
    "reg_alpha": [0, 0.1],  # L1 정규화
    "reg_lambda": [0, 0.1],  # L2 정규화
}

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit=True)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

models["XGBoost"] = grid_search.best_estimator_

y_prob = grid_search.predict_proba(X_test)
y_pred = grid_search.predict(X_test)

"""print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob[:, 1]))"""

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print(f1)

In [ ]:
#LGBMRegressor

model = LGBMRegressor(learning_rate=0.1,
                      data_sample_strategy="goss",
                      top_rate=0.2,
                      other_rate=0.1,
                      force_col_wise=True,
                      verbosity=0,
                      n_jobs=-1,
                      random_state=random_state)


# Define the hyperparameters and their possible values
param_grid = {
    "n_estimators": [25, 50],
    "reg_alpha": [0, 0.1],
    "reg_lambda": [0, 0.1],
    "enable_bundle": [True, False]
}

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit=True)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

models["LightGBM"] = grid_search.best_estimator_

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print(f1)

In [ ]:
# LGBM Classifier
model = LGBMClassifier(learning_rate=0.1,
                      data_sample_strategy="goss",
                      top_rate=0.2,
                      other_rate=0.1,
                      force_col_wise=True,
                      verbosity=0,
                      n_jobs=-1,
                      random_state=random_state)

# Define the hyperparameters and their possible values
param_grid = {
    "n_estimators": [25, 50],
    "reg_alpha": [0, 0.1],
    "reg_lambda": [0, 0.1],
    "enable_bundle": [True, False]
}

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit=True)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

models["LightGBM"] = grid_search.best_estimator_

y_prob = grid_search.predict_proba(X_test)
y_pred = grid_search.predict(X_test)

"""print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob[:, 1]))"""

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print(f1)

In [ ]:
#Neuron Network
import numpy as np
import tensorflow as tf
import random
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# 랜덤 시드 고정
seed_value = 100
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
random.seed(seed_value)

# 신경망 모델 생성 함수
def create_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    return model

# 교차 검증 수행 및 가장 높은 F1 스코어를 가진 모델 선택
f1_scores = []
models1 = []

for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    model = create_model()
    model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=128, verbose=0) #파라미터 값에 따라 정확도 달라짐

    y_pred_prob = model.predict(X_val_fold)
    y_pred = (y_pred_prob > 0.5).astype(int)
    f1 = f1_score(y_val_fold, y_pred)

    f1_scores.append(f1)
    models1.append(model)

best_model_index = np.argmax(f1_scores)
mean_f1_score = np.mean(f1_scores)
best_model = models1[best_model_index]

#K-fold로 여러 모델의 검증 점수를 평균 내어 best 모델을 select
models["Neuron Network"] = best_model

print(f'Best F1 Score: {f1_scores[best_model_index]}')
print(f'Mean F1 Score: {mean_f1_score}')

In [ ]:
#서포트 벡터 머신(Support Vector Machine, SVM) - L2 규제 내장되어있음
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

model = SVC(kernel='rbf', C=1.0)  # RBF 커널을 사용하는 SVM 모델, C는 규제 매개변수

model.fit(X_train, y_train)

models["SVM"]=model

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print(f1)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# 모델 정의
model = MLPClassifier(hidden_layer_sizes=(40, 30),
                      max_iter=400,
                      activation='relu',
                      solver='adam',
                      batch_size=200,
                      learning_rate='invscaling',
                      learning_rate_init=0.01,
                      power_t=0.5,
                      warm_start=True,
                      random_state=100,
                      alpha = 0.01,
                      verbose=True)


# Fit the model
model.fit(X_train, y_train)

# Access the loss_curve_ attribute
loss_values = model.loss_curve_

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print(f1)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Polynomial Regression 모델 생성
model = make_pipeline(PolynomialFeatures(), LinearRegression(n_jobs=-1))

#model.named_steps['polynomialfeatures'].degree = 3  # 다항식의 차수를 3으로 설정

order = 3
# PolynomialFeatures 단계의 order 매개변수를 설정합니다.
model.named_steps['polynomialfeatures'].order = 'C'

"""# Define the hyperparameters and their possible values
param_grid = {
    'polynomialfeatures__degree': [2, 3]  # 다항식 차수
}

# GridSearchCV를 사용하여 최적의 모델 및 하이퍼파라미터 찾기
grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

# 최적의 모델 및 하이퍼파라미터 출력
print("Best parameters: ", grid_search.best_params_)
print("Best CV score: {:.6f}".format(grid_search.best_score_))

# 테스트 데이터에 대해 최적의 모델 사용하여 예측
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)
0.5246142542248348
"""
model.fit(X_train, y_train)
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)

f1_r = f1_score(y_test, predictions)
print(f1_r)

mlp 0.517
* 0.5249
* 0.5323 : learning rate : 0.01, alpha = 0.01 적합

In [ ]:
"""
# 예측값이 담긴 리스트나 배열을 판다스 데이터프레임으로 변환
predictions_df = pd.DataFrame({'y_pred': y_pred})

# 2. 테스트 데이터의 'id' 열을 가져와 예측값과 결합하여 하나의 데이터프레임으로 만듦
result_data = pd.read_csv('test.csv')  # 테스트 데이터 불러오기
result = pd.concat([result_data['id'], predictions_df], axis=1)
# 'id'와 예측값을 결합하여 하나의 데이터프레임으로 만듦
# axis=1 : 열 방향 결합

# 3. 결과를 CSV 파일로 저장
result.to_csv('result.csv', index=False)  # index=False로 설정하여 인덱스를 CSV 파일에 저장하지 않음

"""